In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import joblib

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import joblib

# Step 1: Load the JSON data into a pandas DataFrame
data = pd.read_json('/content/nigeria_fintech_with_scaled_credit_scores.json')

# Step 2: Preprocessing
# Here we assume "Credit Score" is the target variable
X = data.drop("Credit Score", axis=1)  # Features (exclude Credit Score)
y = data["Credit Score"]  # Target (Credit Score)

# Optionally scale the features (important for some models)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 4: Train the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Step 5: Save the trained model to a file
xgb_model.save_model('/content/xgboost_credit_score_model.dat')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:43:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:43:30] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [5]:
import pandas as pd

# Step 1: Load the JSON data into a pandas DataFrame
data = pd.read_json('/content/nigeria_fintech_with_scaled_credit_scores.json')

# Step 2: Extract features (excluding the target column "Credit Score")
# Assuming "Credit Score" is the target variable that you want to exclude
features = data.drop(columns=['Credit Score'])

# Step 3: Save the features as a TSV file
features.to_csv('/content/nigeria_fintech_features.tsv', sep='\t', index=False, header=False)

# Optional: Display the first few rows to verify
print("TSV file content preview:")
print(features.head())

TSV file content preview:
   Account Balance  Wallet Balance (ETH)  Total Mobile Money Transactions  \
0          2694.11                  0.43                           394.31   
1          1760.98                  1.02                           551.74   
2          4881.45                  1.69                           984.22   
3          2291.93                  0.52                           294.28   
4          3063.27                  1.60                           405.42   

   Social Media Activity Data  Mobile Phone Usage  Number of Late Payments  
0                        1497               45.44                        9  
1                        1802               39.45                        7  
2                        3681               69.53                        5  
3                        4960               47.82                        1  
4                        4193               23.67                        4  


In [6]:
!pip install zkml

In [7]:
import math
import os
import unittest

import joblib
import pandas as pd

from zkml.XGBoost.xgboost_to_noir import xgboost_noir_code
from zkml.quantization.quantize import calc_scale, calc_zero_point

class TestXGBoost(unittest.TestCase):
    def test_xgboost_to_noir(self):
        # Quantize part
        q_type = "uint8"

        # Step 1: Load the preprocessing data
        # Assuming you've already uploaded 'nigeria_fintech_features.tsv' to Google Colab
        path = '/content/nigeria_fintech_features.tsv'
        titanic = pd.read_table(path, sep="\t", header=None)
        titanic = titanic.iloc[:, :-1]

        x = [element for sublist in titanic.values.tolist() for element in sublist]

        # Calculate scale and zero point for quantization
        scale_molecule, scale_denominator = calc_scale(x, q_type)
        q_scale = math.ceil(scale_molecule / scale_denominator)
        q_zero_point = calc_zero_point(x, q_scale, q_type)
        print("Quantization info: ", q_scale, q_zero_point)

        # Model to noir code
        # Upload your model to Google Colab and use the correct path
        # Here, we assume the model is a regression model saved as 'xgboost_model.dat'
        model_path = '/content/xgboost_model.dat'  # Adjust this path if needed
        is_classification = "classification" in model_path

        # Load the xgboost model
        clf = joblib.load(model_path)
        noir_code = xgboost_noir_code(clf, q_scale, q_zero_point, q_type, is_classification)

        # Save the noir code to a file
        output_path = '/content/nigeria_fintech.noir'
        with open(output_path, "w") as f:
            for line in noir_code:
                f.write(line)

        print(f"Noir code has been saved to {output_path}")

# Run the test
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


ModuleNotFoundError: No module named 'zkml.XGBoost'